# CPC802: Tópicos Especiais em Inteligência Computacional

A proposta do seguinte código é analisar um conjunto de eventos do protocolo HTTP. Essa análise tem por objetivo treinar um classificador para otimizar o tempo de um analisador de protocolo completo, ou seja, treinaremos o classificador para determinar quando um evento deve ser analisado de forma completa ou não.

Cabe lembrar que existem pesos diferentes:
- falso-positivo (não é um evento malicioso mas foi marcado para envio ao analisador de protocolo): só estamos desperdiçando tempo do analisador (isso não é um problema).
- falso-negativo (é um evento malicioso mas foi marcado para não enviar ao analisador de protocolo): devemos evitar esse comportamento pois nesse caso deixaremos um evento malicioso sem ser analisado.

O presente código analisa as URLs separando elas em **tokens** ou **caracteres**. Mais detalhes estão na função `getTokens`.


In [0]:
################################################################################
# Configurações do código
################################################################################

# Utilização de tokens/words ou de caracteres. Caso seja True, usa words, caso seja False, utiliza caracteres.
use_words = True

# Expressão regular utilizada como separador das URLs.
tokensSep = '/|-|\.|=|&|\?|\s+|\<|\>|;|\(|\)'

# Utilização dos arquivos:
fileForTrainingBad = 'badqueries.txt'
fileForTrainingGood = 'goodqueries.txt'

fileForValidationBad = 'badqueries_gg.txt'
fileForValidationGood = 'goodqueries_gg.txt'

# Sufixo para escrita do nome do arquivo
saveSuffix = '-word'

In [0]:
# Carregamos as bibliotecas.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [0]:
import os
import urllib.parse

# Função que lê os dados do arquivo.
def loadFile(name):
    # Obtém diretório e nome completo do arquivo (com path).
    directory = str(os.getcwd())
    filepath = os.path.join(directory, name)

    # Faz a leitura das linhas do arquivo.
    with open(filepath,'r') as f:
        data = f.readlines()

    # Transforma as linhas em um Set (retira as duplicadas) e depois em uma lista.
    data = list(set(data))

    # Realiza o decode das URLs e coloca elas na lista result.
    result = []
    for d in data:
        d = str(urllib.parse.unquote(d))
        result.append(d)

    return result

In [0]:
import re

# Função que separa cada URL em uma lista de palavras/tokens, utilizando como separadores: '/', '-', '.'
def getTokens(input):
    return re.split(tokensSep, str(input.encode('utf-8')))

# Exemplo:
# getTokens('/wikipedia/noticias/museu-nacional-e-10.php?paramenter=10&c=select * from table&opa=<xss(alert)>')

Vamos fazer a leitura dos dados do Google Drive (estamos executando no Google Colab para validação)

In [183]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [184]:
%cd drive/'My Drive'/'Colab Notebooks'/cpc802

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/cpc802'
/content/drive/My Drive/Colab Notebooks/cpc802


In [0]:
# Criamos uma instância de dicionário para converter as URLs em representações
#   numéricas de acordo com a frequência de utilização deles.
if use_words:
    vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="word", sublinear_tf=True, tokenizer=getTokens)
else:
    vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3))

In [0]:
# Carrega os dados dos arquivos
badQueries = loadFile(fileForTrainingBad)
validQueries = loadFile(fileForTrainingGood)

allQueries = badQueries + validQueries

In [0]:
# Geramos o vetor de conversão para as URLs.
X = vectorizer.fit_transform(allQueries)

In [0]:
# Vamos colocar os labels
#   0 - normal
#   1 - malicious
yBad = [1 for i in range(0, len(badQueries))]
yGood = [0 for i in range(0, len(validQueries))]

y = yBad + yGood

# Vamos gerar os dados de treinamento e validação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [189]:
badCount = len(badQueries)
badCount

44713

In [190]:
validCount = len(validQueries)
validCount

1265994

In [0]:
# Criamos um classificador usando regressão logística.
lgs = LogisticRegression(class_weight='balanced', max_iter=400)

In [192]:
# Treinando o modelo.
lgs.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=400, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [193]:
# Salva modelo treinado no Google Drive.
import joblib
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = 'cpc802-' + timestr + saveSuffix + '.sav'
filenameVectorizer = 'cpc802-' + timestr + saveSuffix +'.vectorizer'
joblib.dump(lgs, filename)
joblib.dump(vectorizer, filenameVectorizer)

['cpc802-20200229-234322-word.vectorizer']

In [194]:
# Validamos o modelo
predicted = lgs.predict(X_test)

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)
print("Bad samples: %d" % badCount)
print("Good samples: %d" % validCount)
print("Baseline Constant negative: %.6f" % (validCount / (validCount + badCount)))
print("Accuracy: %f" % lgs.score(X_test, y_test))
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

Bad samples: 44713
Good samples: 1265994
Baseline Constant negative: 0.965886
Accuracy: 0.996353
Precision: 0.930887
Recall: 0.963169
F1-Score: 0.946753
AUC: 0.990835


In [195]:
# Testando com valores conhecidos.
X_predict = [
             '/busca/manoel',
             '/search=faizanahad',
             '/getpassword.php',
             '/wp-admin/includes/log.exe',
             '/nethost.exe',
             '/centroesteticosothys/img/_notes/gum.exe?c=select * from table',
             '/node/add',
             ]
X_predict = vectorizer.transform(X_predict)
y_Predict = lgs.predict_log_proba(X_predict)
print(y_Predict)
y_Predict = lgs.predict(X_predict)
print(y_Predict)

[[-4.91809773e-02 -3.03673808e+00]
 [-1.11225812e-01 -2.25129029e+00]
 [-5.16207267e-02 -2.98953134e+00]
 [-7.43347875e-02 -2.63611340e+00]
 [-4.81886957e-02 -3.05662841e+00]
 [-6.40222893e+00 -1.65923366e-03]
 [-5.99172427e-02 -2.84460000e+00]]
[0 0 0 0 0 1 0]


In [0]:
# Prepara dataset real apenas para validação.
otherBadQueries = loadFile(fileForValidationBad)
otherGoodQueries = loadFile(fileForValidationGood)
allOtherQueries = otherBadQueries + otherGoodQueries

# Vetoriza dataset real.
X_real = vectorizer.transform(allOtherQueries)

# Calcula as inferências do dataset real.
y_real = lgs.predict(X_real)

# Vamos colocar os labels no dataset real.
#   0 - normal
#   1 - malicious
yBad_gg = [1 for i in range(0, len(otherBadQueries))]
yGood_gg = [0 for i in range(0, len(otherGoodQueries))]
y_real_with_label = yBad_gg + yGood_gg


In [197]:
# Verifica acertos e erros de predição usando dataset real.
acertos = 0
erros = 0
for i in range(len(y_real)):
    if y_real[i] == y_real_with_label[i]:
        acertos = acertos + 1
    elif (y_real_with_label[i] == 0 and y_real[i] == 1):
        acertos = acertos + 1
    else:
        erros = erros + 1

acertos/len(y_real)

0.9099784663673857

In [198]:
# Verifica acertos e erros de predição usando dataset de treino (é para verificar a acurácia).
#   Nessa verificação levamos em conta os erros que não prejudicam a avaliação do domínio
#   do problema, ou seja, quando o classificador aponta que uma URL é maliciosa e ela
#   não é.

# Vetoriza dataset usado no treinamento.
X_trained = vectorizer.transform(allQueries)

# Calcula as inferências do dataset real.
y_trained = lgs.predict(X_trained)

acertos = 0
erros = 0
for i in range(len(y_trained)):
    if y_trained[i] == y[i]:
        acertos = acertos + 1
    elif (y[i] == 0 and y_trained[i] == 1):
        acertos = acertos + 1
    else:
        erros = erros + 1

acertos/len(y_trained)

0.999735257383992